In [1]:
import nltk
import random
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import csv
from nltk.corpus import stopwords
import string
import emoji

In [2]:
with open('dataset/apple-sentiment.csv', 'r') as nodecsv: # Open the file                       
    csvreader = csv.reader(nodecsv) # Read the csv  
    # Retrieve the data (using Python list comprhension and list slicing to remove the header row)
    csv = [n for n in csvreader][1:]

In [28]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes 
        \S*
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>')# HTML tags
regex_str.append(r'(?:@[\w_]+)')# @-mentions
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [29]:
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', '…','•']

In [30]:
for l in stop:
    print(l)

i
me
my
myself
we
our
ours
ourselves
you
you're
you've
you'll
you'd
your
yours
yourself
yourselves
he
him
his
himself
she
she's
her
hers
herself
it
it's
its
itself
they
them
their
theirs
themselves
what
which
who
whom
this
that
that'll
these
those
am
is
are
was
were
be
been
being
have
has
had
having
do
does
did
doing
a
an
the
and
but
if
or
because
as
until
while
of
at
by
for
with
about
against
between
into
through
during
before
after
above
below
to
from
up
down
in
out
on
off
over
under
again
further
then
once
here
there
when
where
why
how
all
any
both
each
few
more
most
other
some
such
no
nor
not
only
own
same
so
than
too
very
s
t
can
will
just
don
don't
should
should've
now
d
ll
m
o
re
ve
y
ain
aren
aren't
couldn
couldn't
didn
didn't
doesn
doesn't
hadn
hadn't
hasn
hasn't
haven
haven't
isn
isn't
ma
mightn
mightn't
mustn
mustn't
needn
needn't
shan
shan't
shouldn
shouldn't
wasn
wasn't
weren
weren't
won
won't
wouldn
wouldn't
!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
:
;
<
=
>
?
@
[
\
]
^
_
`
{
|
}
~


In [31]:
def tokenize(s):
    tokens = tokens_re.findall(s)
    return tokens

In [33]:
def cleanTweet(token,regex):
    terms_all = [emoji.demojize(term) for term in token if term.lower() not in stop and not regex.match(term)]
    return terms_all

In [34]:
tokens = []
for c in csv:
    tokens.append(tokenize(c[0]))

In [35]:
print(tokens[:5])

[['Global', 'warming', 'report', 'urges', 'governments', 'to', 'act', '|', 'BRUSSELS', ',', 'Belgium', '(', 'AP', ')', '-', 'The', 'world', 'faces', 'increased', 'hunger', 'and', '.', '.', '[', 'link', ']'], ['Fighting', 'poverty', 'and', 'global', 'warming', 'in', 'Africa', '[', 'link', ']'], ['Carbon', 'offsets', ':', 'How', 'a', 'Vatican', 'forest', 'failed', 'to', 'reduce', 'global', 'warming', '[', 'link', ']'], ['Carbon', 'offsets', ':', 'How', 'a', 'Vatican', 'forest', 'failed', 'to', 'reduce', 'global', 'warming', '[', 'link', ']'], ['URUGUAY', ':', 'Tools', 'Needed', 'for', 'Those', 'Most', 'Vulnerable', 'to', 'Climate', 'Change', '[', 'link', ']']]


In [44]:
import operator

exclude_str = []
exclude_str.append(emoticons_str)
exclude_str.append(r'<[^>]+>')# HTML tags
exclude_str.append(r'(?:@[\w_]+)')# @-mentions
exclude_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
exclude_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
exclude_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers

exclude_re = re.compile(r'('+'|'.join(exclude_str)+')', re.VERBOSE | re.IGNORECASE)
all_word=[]
from collections import Counter
count_all = Counter()
for token in tokens:
    cleanToken = cleanTweet(token,exclude_re)
    terms_all = [term.lower() for term in cleanToken]
    count_all.update(terms_all)
    for w in terms_all:
        all_word.append(w)
print(count_all.most_common(100))

[('climate', 3422), ('change', 3301), ('global', 3121), ('warming', 3057), ('link', 978), ('new', 334), ('bill', 247), ('snow', 239), ('news', 223), ('science', 197), ('report', 179), ('energy', 179), ('us', 173), ('world', 158), ('à_', 146), ('u', 144), ('dc', 141), ('day', 138), ('could', 136), ('earth', 135), ('obama', 133), ('scientists', 131), ('green', 130), ('al', 120), ('gore', 117), ('conference', 110), ('fight', 109), ('weather', 108), ('un', 108), ('may', 107), ('senate', 106), ('think', 106), ('one', 104), ('legislation', 103), ('says', 101), ('get', 101), ('say', 101), ('time', 101), ('carbon', 96), ('believe', 96), ('com', 95), ('washington', 94), ('help', 94), ('great', 93), ('agency', 91), ('immigration', 90), ('people', 89), ('law', 88), ('winter', 87), ('worse', 86), ('good', 83), ('april', 82), ('action', 81), ('due', 80), ('cold', 80), ('like', 79), ('stop', 78), ('still', 78), ('take', 77), ('study', 76), ('make', 73), ('p', 73), ('makes', 72), ('air', 71), ('debat

In [42]:
text="Photo: Love IOS 8 @apple @iphone #Love #IOS8 #NoCrop #iphone #young #phone #screen #capturescream #boy... http://t.co/fIDj9FxHDV"
token=tokenize(text)
cleanToken=cleanTweet(token,exclude_re)

In [39]:
print(text)

Photo: Love IOS 8 @apple @iphone #Love #IOS8 #NoCrop #iphone #young #phone #screen #capturescream #boy... http://t.co/fIDj9FxHDV


In [40]:
print(token)

['Photo', ':', 'Love', 'IOS', '8', '@apple', '@iphone', '#Love', '#IOS8', '#NoCrop', '#iphone', '#young', '#phone', '#screen', '#capturescream', '#boy', '.', '.', '.', 'http://t.co/fIDj9FxHDV']


In [43]:
print(cleanToken)

['Photo', 'Love', 'IOS', '@apple', '@iphone']


In [45]:
nltk.download('averaged_perceptron_tagger')
pos = nltk.pos_tag(all_word)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
#  j = adjective, r = adverb, n=noun and v = verb
allowed_word_types = ["N","J","V"]
all_adj=[w[0] for w in pos if w[1][0] in allowed_word_types]

In [47]:
allwords = nltk.FreqDist(all_adj)

In [16]:
word_features = list(allwords.keys())[:5000]

In [48]:
print(word_features)

['global', 'warming', 'report', 'urges', 'governments', 'act', 'brussels', 'belgium', 'ap', 'world', 'faces', 'increased', 'hunger', 'link', 'fighting', 'poverty', 'africa', 'carbon', 'offsets', 'vatican', 'forest', 'failed', 'reduce', 'uruguay', 'tools', 'needed', 'vulnerable', 'climate', 'change', 'ocean', 'saltiness', 'shows', 'intensifying', 'water', 'cycle', 'evidence', 'message', 'deniers', 'clueless', 'kennedy', 'flies', 'pri', 'physicists', 'violates', 'physics', 'reveal', 'uab', 'clarendon', 'shirt', 'laughs']


In [50]:
#document = [(w[0],w[1]) for w in csv]
document=[]
for w in csv:
    categ=""
    if w[1] in ['Yes','Y','5']:
        categ="pos"
        document.append((w[0],categ))
    if w[1] in ['No','N','1']:
        categ="neg"
        document.append((w[0],categ))

In [51]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [52]:
featuresets = [(find_features(rev), category) for (rev, category) in document]

In [53]:
print(featuresets[:1])

[({'global': False, 'warming': True, 'report': True, 'urges': True, 'governments': True, 'act': False, 'brussels': False, 'belgium': False, 'ap': False, 'world': True, 'faces': True, 'increased': True, 'hunger': True, 'link': True, 'fighting': False, 'poverty': False, 'africa': False, 'carbon': False, 'offsets': False, 'vatican': False, 'forest': False, 'failed': False, 'reduce': False, 'uruguay': False, 'tools': False, 'needed': False, 'vulnerable': False, 'climate': False, 'change': False, 'ocean': False, 'saltiness': False, 'shows': False, 'intensifying': False, 'water': False, 'cycle': False, 'evidence': False, 'message': False, 'deniers': False, 'doubters': False, 'look': False, 'migratory': False, 'birds': False, 'new': False, 'strategy': False, 'stay': False, 'southern': False, 'competing': False, 'limpopo': False, 'bring': False, 'higher': False, 'temperatures': False, 'southe': False, 'impact': False, 'wheat': False, 'rice': False, 'production': False, 'india': False, 'ludhian

In [54]:
# menyusun data training dan testing 
trainsize = round(len(featuresets)*0.7)

random.shuffle(featuresets)
training_set = featuresets[:trainsize]
testing_set = featuresets[trainsize:]

In [58]:
sum(p[1]=='neg' for p in training_set)

769

In [59]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(25)

Classifier accuracy percent: 76.87450670876085
Most Informative Features
                 hearing = True              neg : pos    =     16.1 : 1.0
                 control = True              neg : pos    =     16.1 : 1.0
                imminent = True              neg : pos    =     14.2 : 1.0
                   snows = True              neg : pos    =     14.2 : 1.0
                disaster = True              neg : pos    =     12.3 : 1.0
                  shovel = True              neg : pos    =     12.3 : 1.0
                happened = True              neg : pos    =     12.3 : 1.0
               alarmists = True              neg : pos    =     12.3 : 1.0
                      al = True              neg : pos    =     11.9 : 1.0
                    hoax = True              neg : pos    =     11.8 : 1.0
                    gore = True              neg : pos    =     10.8 : 1.0
                   stuff = True              neg : pos    =     10.4 : 1.0
               shoveling = 

In [24]:
from nltk.metrics.scores import (precision, recall)
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print('Recall:', recall(refsets['pos'], testsets['pos']))
print('Precision:', precision(refsets['pos'], testsets['pos']))

Recall: 0.8728260869565218
Precision: 0.8373305526590198


In [41]:
classifier2 = nltk.classify.SklearnClassifier(LinearSVC())
classifier2.train(training_set)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [42]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier2, testing_set))*100)

Classifier accuracy percent: 81.29439621152328


In [43]:
classifier3 = SklearnClassifier(LogisticRegression(solver='lbfgs')).train(training_set)

In [44]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier3, testing_set))*100)

Classifier accuracy percent: 81.05761641673243


In [45]:
for (rev, category) in document[trainsize:]:
    result=classifier2.classify(find_features(rev))
    print(rev,"-",category,"-",result)

US Senate CAP & TAX *** Bill to be unveiled April 26 *** Global Warming Swindle (Video's too)|Our work is cut out .. http://oohja.com/xb3rj - neg - neg
Algae for controlling global warming - Current News & Events ... http://bit.ly/ax6lEp - pos - pos
Addressing Global Warming: Retrain Coal Workers for Green Jobs ... http://bit.ly/cW1KTV - pos - pos
Forests Are Growing Faster,Climate Change Appears to Be Driving Accelerated Growth (Science Daily) http://bit.ly/d1EqlN @InvasiveNotes - pos - pos
Obama's base cares about their religion of global warming and Marxism. The myth of healthcare. The myth of peace without having to be strong - neg - neg
Global warming may become global cooling this century http://bit.ly/cU8D4F - pos - pos
Global Warming: Ozone Hole Healing http://bit.ly/aXpdcl - pos - pos
My daughter says Global Warming Is NOT Cool!  http://twitgoo.com/ok5ya - pos - pos
Earth's polar ice sheets vulnerable to even moderate global warming; New Orleans, much of southern F... http://c